In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv("BANPUsecond1522.csv")
dt=data
data.drop(data.columns[[0]],axis=1,inplace=True)
window_size = int(len(data)/(8))
splittrain = []
splittest = [] 
for i in range(0, len(data)-4*window_size,window_size):
    splittrain.append(data[i:i+3*window_size])
    splittest.append(data[i+3*window_size:i+5*window_size])

In [2]:
from scipy.optimize import NonlinearConstraint, Bounds, LinearConstraint
import sympy as sp
w1=sp.symbols('w1')
w2=sp.symbols('w2')
w3=sp.symbols('w3')
w4=sp.symbols('w4')
w5=sp.symbols('w5')
w6=sp.symbols('w6')
w7=sp.symbols('w7')
w8=sp.symbols('w8')
w9=sp.symbols('w9')
w10=sp.symbols('w10')
tdbuy=sp.symbols('tdbuy')
tdsell=sp.symbols('tdsell')

In [3]:
def const(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,tdbuy,tdsell = p
    return w1+w2+w3+w4+w5+w6+w7+w8+w9+w10
nlc = NonlinearConstraint(const, 1 ,1)
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint, Bounds

bounds = [[0, 1], [0, 1],[0, 1], [0, 1],[0, 1],[0, 1],[0, 1], [0, 1],[0, 1],[0,1],[0.2,1],[0.2,1]]

In [4]:
coll,buy_idx,sell_idx=[],[],[]
count1,count2,count3=0,0,0
def func(p):
    global S,B,resl,resfl
    S=0
    B=0
    resl=[]
    resfl=[]
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,tdbuy,tdsell = p
    mon=100000
    amount=0
    for i in range(0,len(dfo)):
        r = ((w1*sl1[i]+w2*sl2[i]+w3*sl3[i]+w4*sl4[i]+w5*sl5[i]+w6*sl6[i]+w7*sl7[i]+w8*sl8[i]+w9*sl9[i]+w10*sl10[i])/(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10))#w1=sp.symbols('w1')
        '''if i!=0:
            if dfo['adj_close_price'][i]<dfo['adj_close_price'][i-1]*0.8:
                mon+=amount*dfo['adj_close_price'][i]*0.998
                amount=0'''
        if r>tdbuy:
            sumall=mon//dfo['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*dfo['adj_close_price'][i]*1.002
            B+=1
        elif r<-tdsell:
            mon+=amount*dfo['adj_close_price'][i]*0.998
            amount=0
            S+=1
        else:
            pass
    ret=mon+amount*dfo['adj_close_price'][len(dfo)-1]
    return -((ret-100000)/1000)
def fobj():
    global sl1,sl2,sl3,sl4,sl5,sl6,sl7,sl8,sl9,sl10,dfo,decision_d_l,bh_test_l,bh_train_l,train_l,test_l
    bh_test_l,bh_train_l,train_l,test_l,tdbuy_l,tdsell_l,buy_l,sell_l=[],[],[],[],[],[],[],[]
    global count1,count2,count3,sell_idx,buy_idx
    count1,count2,count3=0,0,0
    for j in range(len(splittrain)):
        dfo=splittrain[j]
        dfo=dfo.reset_index()
        sl1=dfo['volume_prof_signal']
        sl2=dfo['macd_crossover_signal']
        sl3=dfo['psar_adx_signal']
        sl4=dfo['renko_adx_signal']
        sl5=dfo['aroon_kp_signal']
        sl6=dfo['macd_overzero_signal']
        sl7=dfo['ma5_10_signal']
        sl8=dfo['cdc_signal']
        sl9=dfo['bb_rsi_signal']
        sl10=dfo['rsi_adx_signal']
        result = differential_evolution(func, bounds,constraints=(nlc),maxiter=2500,popsize=90,recombination=0.8) 
        coll.append([result.x,-result.fun,j])
        print([result.x,-result.fun,j])
        train_l.append(-result.fun)
        a=result.x
        b=j
        tdbuy=a[10]
        tdsell=a[11]
        w=a[0:10]
        df=splittest[b]
        decision_d_l=[]
        il=[]
        
        mon=100000
        amount=0
        sumall=0
        df=df.reset_index()
        sl1=df['volume_prof_signal']
        sl2=df['macd_crossover_signal']
        sl3=df['psar_adx_signal']
        sl4=df['renko_adx_signal']
        sl5=df['aroon_kp_signal']
        sl6=df['macd_overzero_signal']
        sl7=df['ma5_10_signal']
        sl8=df['cdc_signal']
        sl9=df['bb_rsi_signal']
        sl10=df['rsi_adx_signal']

        for i in range(0,len(df)):
            decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
            '''if i!=0:
                if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                        mon+=amount*df['adj_close_price'][i]*0.998
                        amount=0
                        sell_idx.append(i)'''
            if decision_d>tdbuy:
                sumall=mon//df['adj_close_price'][i]
                amount+=sumall
                mon-=sumall*df['adj_close_price'][i]*1.002
                count1+=1
                buy_idx.append(i)
            elif decision_d<-tdsell:
                mon+=amount*df['adj_close_price'][i]*0.998
                amount=0
                count2+=1
                sell_idx.append(i)
            else:
                count3+=1
            decision_d_l.append(decision_d)
        il.append(i)
        ret=mon+amount*df['adj_close_price'][len(df)-1]
        stocknum=100000/dfo['adj_close_price'][0]
        bh_test_l.append(((stocknum*(0.998*df['adj_close_price'][len(df)-1])+(100000-stocknum*df['adj_close_price'][0]*1.002))-100000)/1000)
        bh_train_l.append(((stocknum*(0.998*dfo['adj_close_price'][len(dfo)-1])+(100000-stocknum*dfo['adj_close_price'][0]*1.002))-100000)/1000)
        test_l.append((ret-100000)/1000)
        tdbuy_l.append(tdbuy)
        tdsell_l.append(tdsell)
        buy_l.append(count1)
        sell_l.append(count2)
        print("B&H_train :",bh_train_l)
        print("ATS_train buy:{b} sell:{s} return:{r}".format(b = B,s = S,r=train_l))
        print("B&H_test :",bh_test_l)
        print("tdbuy:{b} tdsell:{s}".format(b=tdbuy_l,s=tdsell_l))
        print("buy:{b} sell:{s}".format(b=buy_l,s=sell_l))
        print("ATS_test return:{r}".format(r=test_l))

In [5]:
fobj() #new

C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.13391246, 0.05903299, 0.04382747, 0.05449629, 0.08783919,
       0.07169942, 0.19753022, 0.03811175, 0.0681533 , 0.24539691,
       0.33214216, 0.35211505]), 75.92635206769368, 0]
B&H_train : [17.18131154060777]
ATS_train buy:4 sell:0 return:[75.92635206769368]
B&H_test : [-40.661091860154194]
tdbuy:[0.3321421587504799] tdsell:[0.35211505107695007]
buy:[1] sell:[3]
ATS_test return:[-24.767289218383798]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.02485075, 0.12943515, 0.53428611, 0.00234388, 0.00569496,
       0.03301168, 0.06544987, 0.07113109, 0.03787732, 0.0959192 ,
       0.20846648, 0.49457024]), 81.32849548795603, 1]
B&H_train : [17.18131154060777, 41.056105265661344]
ATS_train buy:7 sell:3 return:[75.92635206769368, 81.32849548795603]
B&H_test : [-40.661091860154194, -25.6456310450974]
tdbuy:[0.3321421587504799, 0.20846647522372658] tdsell:[0.35211505107695007, 0.49457023674258593]
buy:[1, 6] sell:[3, 4]
ATS_test return:[-24.767289218383798, -13.974212401219367]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.13235524, 0.06637077, 0.06321142, 0.45699961, 0.06507805,
       0.14140488, 0.00646891, 0.01772536, 0.00103592, 0.04934984,
       0.3239336 , 0.20890192]), 26.73337270087436, 2]
B&H_train : [17.18131154060777, 41.056105265661344, -30.972715992480094]
ATS_train buy:5 sell:11 return:[75.92635206769368, 81.32849548795603, 26.73337270087436]
B&H_test : [-40.661091860154194, -25.6456310450974, 1.5710161241863534]
tdbuy:[0.3321421587504799, 0.20846647522372658, 0.32393360312765185] tdsell:[0.35211505107695007, 0.49457023674258593, 0.20890192200760627]
buy:[1, 6, 43] sell:[3, 4, 9]
ATS_test return:[-24.767289218383798, -13.974212401219367, 62.88731946919815]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.01279889, 0.0016082 , 0.0373057 , 0.00766501, 0.63354257,
       0.00103165, 0.16385472, 0.01679039, 0.1225202 , 0.00288268,
       0.81039315, 0.67334857]), 95.40332388961792, 3]
B&H_train : [17.18131154060777, 41.056105265661344, -30.972715992480094, -33.47598156397891]
ATS_train buy:1 sell:6 return:[75.92635206769368, 81.32849548795603, 26.73337270087436, 95.40332388961792]
B&H_test : [-40.661091860154194, -25.6456310450974, 1.5710161241863534, 20.67182314021242]
tdbuy:[0.3321421587504799, 0.20846647522372658, 0.32393360312765185, 0.8103931463787812] tdsell:[0.35211505107695007, 0.49457023674258593, 0.20890192200760627, 0.6733485722943408]
buy:[1, 6, 43, 44] sell:[3, 4, 9, 14]
ATS_test return:[-24.767289218383798, -13.974212401219367, 62.88731946919815, -3.202526753215774]


In [6]:
fobj()

C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.06492053, 0.01203352, 0.1283422 , 0.02103918, 0.01863431,
       0.02216128, 0.20763864, 0.02004935, 0.06674466, 0.43843633,
       0.56707804, 0.91183503]), 75.62565359579469, 0]
B&H_train : [17.18131154060777]
ATS_train buy:1 sell:0 return:[75.62565359579469]
B&H_test : [-40.661091860154194]
tdbuy:[0.5670780376976856] tdsell:[0.9118350273607939]
buy:[0] sell:[0]
ATS_test return:[0.0]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.09704207, 0.20345485, 0.00321673, 0.46752077, 0.0867248 ,
       0.00118267, 0.0325676 , 0.05227826, 0.05371512, 0.00229714,
       0.54949721, 0.38108694]), 83.35156727009202, 1]
B&H_train : [17.18131154060777, 41.056105265661344]
ATS_train buy:1 sell:1 return:[75.62565359579469, 83.35156727009202]
B&H_test : [-40.661091860154194, -25.6456310450974]
tdbuy:[0.5670780376976856, 0.5494972105829438] tdsell:[0.9118350273607939, 0.3810869389848082]
buy:[0, 6] sell:[0, 0]
ATS_test return:[0.0, -23.068069717879304]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.02222158, 0.09172822, 0.00562562, 0.02774939, 0.33100761,
       0.00772319, 0.29827233, 0.08038831, 0.09154025, 0.04374348,
       0.66777106, 0.5298947 ]), 20.343484214126583, 2]
B&H_train : [17.18131154060777, 41.056105265661344, -30.972715992480094]
ATS_train buy:1 sell:4 return:[75.62565359579469, 83.35156727009202, 20.343484214126583]
B&H_test : [-40.661091860154194, -25.6456310450974, 1.5710161241863534]
tdbuy:[0.5670780376976856, 0.5494972105829438, 0.6677710605372245] tdsell:[0.9118350273607939, 0.3810869389848082, 0.5298947048105455]
buy:[0, 6, 8] sell:[0, 0, 5]
ATS_test return:[0.0, -23.068069717879304, 70.09206462767601]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.14091207, 0.01741437, 0.00111265, 0.29238811, 0.08215274,
       0.04511261, 0.10064954, 0.28578729, 0.02982753, 0.00464308,
       0.55031114, 0.76136893]), 95.40332388961792, 3]
B&H_train : [17.18131154060777, 41.056105265661344, -30.972715992480094, -33.47598156397891]
ATS_train buy:1 sell:0 return:[75.62565359579469, 83.35156727009202, 20.343484214126583, 95.40332388961792]
B&H_test : [-40.661091860154194, -25.6456310450974, 1.5710161241863534, 20.67182314021242]
tdbuy:[0.5670780376976856, 0.5494972105829438, 0.6677710605372245, 0.5503111420016852] tdsell:[0.9118350273607939, 0.3810869389848082, 0.5298947048105455, 0.761368927943244]
buy:[0, 6, 8, 8] sell:[0, 0, 5, 5]
ATS_test return:[0.0, -23.068069717879304, 70.09206462767601, 0.0]


In [7]:
fobj()

C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.1254872 , 0.03322859, 0.35990581, 0.02037037, 0.00215533,
       0.36387822, 0.00513015, 0.03655582, 0.05241047, 0.00087804,
       0.47993523, 0.53802396]), 57.63259648988343, 0]
B&H_train : [17.18131154060777]
ATS_train buy:1 sell:1 return:[57.63259648988343]
B&H_test : [-40.661091860154194]
tdbuy:[0.4799352295007462] tdsell:[0.5380239617198983]
buy:[2] sell:[0]
ATS_test return:[-44.084942227197644]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.12104561, 0.06856199, 0.0304713 , 0.13159266, 0.05353278,
       0.00338624, 0.13979882, 0.0333651 , 0.00888875, 0.40935674,
       0.51132224, 0.32705131]), 97.6555614403648, 1]
B&H_train : [17.18131154060777, 41.056105265661344]
ATS_train buy:1 sell:2 return:[57.63259648988343, 97.6555614403648]
B&H_test : [-40.661091860154194, -25.6456310450974]
tdbuy:[0.4799352295007462, 0.5113222370725257] tdsell:[0.5380239617198983, 0.32705131277290633]
buy:[2, 9] sell:[0, 2]
ATS_test return:[-44.084942227197644, 32.787310656850806]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([2.34212839e-02, 1.34542285e-02, 1.37010845e-02, 4.16475762e-01,
       2.64539587e-01, 4.93543943e-02, 8.88607474e-02, 8.58895641e-02,
       4.38494850e-02, 4.53863736e-04, 4.65163520e-01, 3.40974094e-01]), 22.032230787734996, 2]
B&H_train : [17.18131154060777, 41.056105265661344, -30.972715992480094]
ATS_train buy:1 sell:7 return:[57.63259648988343, 97.6555614403648, 22.032230787734996]
B&H_test : [-40.661091860154194, -25.6456310450974, 1.5710161241863534]
tdbuy:[0.4799352295007462, 0.5113222370725257, 0.4651635204301195] tdsell:[0.5380239617198983, 0.32705131277290633, 0.3409740938549803]
buy:[2, 9, 34] sell:[0, 2, 6]
ATS_test return:[-44.084942227197644, 32.787310656850806, 93.00331868726347]


C:\Users\admin\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.02418975, 0.02884164, 0.50277518, 0.01871348, 0.00348018,
       0.12466487, 0.02568917, 0.0159812 , 0.20439499, 0.05126954,
       0.40601302, 0.24337551]), 36.967717190017694, 3]
B&H_train : [17.18131154060777, 41.056105265661344, -30.972715992480094, -33.47598156397891]
ATS_train buy:4 sell:6 return:[57.63259648988343, 97.6555614403648, 22.032230787734996, 36.967717190017694]
B&H_test : [-40.661091860154194, -25.6456310450974, 1.5710161241863534, 20.67182314021242]
tdbuy:[0.4799352295007462, 0.5113222370725257, 0.4651635204301195, 0.40601301888081687] tdsell:[0.5380239617198983, 0.32705131277290633, 0.3409740938549803, 0.24337550758308457]
buy:[2, 9, 34, 35] sell:[0, 2, 6, 11]
ATS_test return:[-44.084942227197644, 32.787310656850806, 93.00331868726347, -17.2388997756958]


In [ ]:
fobj()#new2

In [ ]:
fobj()

In [27]:
# coll=[[[0.08492177, 0.17384463, 0.01297783, 0.02911209, 0.00305128,
#        0.29679029, 0.09341777, 0.03109278, 0.2669813 , 0.00781026,
#        0.15002345], 77.78606595899961, 0],[[0.10124751, 0.07947271, 0.14059461, 0.01134871, 0.07670483,
#        0.0105102 , 0.01523628, 0.0793311 , 0.32688289, 0.15867116,
#        0.16469997], 29.423729452217085, 1],[[0.10424396, 0.04083642, 0.14153184, 0.04832486, 0.02425143,
#        0.23456657, 0.01785653, 0.04702162, 0.1687655 , 0.17260126,
#        0.23068131], 20.744831571363452, 2],[[0.03418246, 0.03985494, 0.05818124, 0.01006821, 0.05736128,
#        0.51578311, 0.04809931, 0.05745843, 0.05766466, 0.12134635,
#        0.13210764], 17.658730539011014, 3],[[0.04441311, 0.00105711, 0.10547514, 0.24048136, 0.00218894,
#        0.03192681, 0.0093497 , 0.09019205, 0.3072222 , 0.16769358,
#        0.2397252 ], 38.12829733082201, 4],[[0.15608604, 0.46143613, 0.02077728, 0.15747731, 0.00652082,
#        0.03187775, 0.00144456, 0.02996531, 0.12415056, 0.01026426,
#        0.05789345], 34.24385590455824, 5],[[0.03761342, 0.34845239, 0.11776582, 0.00239228, 0.03168897,
#        0.02931779, 0.15102881, 0.08606771, 0.11747511, 0.07819771,
#        0.18479926], 27.365418176465987, 6],[[0.03660642, 0.05033639, 0.12932883, 0.07908549, 0.01884995,
#        0.00878074, 0.15192304, 0.25559357, 0.00167098, 0.2678246 ,
#        0.22253394], 48.53843252603145, 7],[[0.00772096, 0.47030788, 0.07248883, 0.04361143, 0.25529928,
#        0.04781215, 0.00984386, 0.00458237, 0.02302817, 0.06530506,
#        0.11147168], 92.85205689248085, 8],[[0.02404885, 0.11613386, 0.04691288, 0.15394344, 0.01931855,
#        0.0351663 , 0.15083233, 0.02116866, 0.27700484, 0.15547028,
#        0.28969039], 78.26595460097695, 9]]

In [146]:
#test
import pandas as pd
sums=0
listna=[]
for j in range(0,11): #เเต่ละ signal 
    for i in range(0,10): #เเต่ละ data
        sums+=coll[i][0][j]
    sums=sums/10
    listna.append(sums)
tdna=listna[10]
w=listna[0:10]
dfo=pd.read_csv('TSLA_1417_year_testing_signal.csv')
count1,count2,count3=0,0,0
decision_d_l=[]
il=[]
sell_idx,buy_idx=[],[]
def testing(td,w,df):
    global count1,count2,count3,sell_idx,buy_idx,sl1
    mon=100000
    amount=0
    sumall=0
    sl1=dfo['sma89_signal']
    sl2=dfo['obv_macd_signal']
    sl3=dfo['cdc_signal']
    sl4=dfo['rsi_signal']
    sl5=dfo['renko_signal']
    sl6=dfo['sar_signal']
    sl7=dfo['macd_signal']
    sl8=dfo['stoch_signal']
    sl9=dfo['obv_renko_signal']
    sl10=dfo['aroon_signal']
    
    for i in range(0,len(df)):
        decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
        if i!=0:
            if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
                    sell_idx.append(i)
        if decision_d>td:
            sumall=mon//df['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*df['adj_close_price'][i]*1.002
            count1+=1
            buy_idx.append(i)
        elif decision_d<-td:
            mon+=amount*df['adj_close_price'][i]*0.998
            amount=0
            count2+=1
            sell_idx.append(i)
        else:
            count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
    ret=mon+amount*df['adj_close_price'][len(df)-1]
    return (ret-100000)/1000
testing(tdna,w,dfo)

8.7122626862755

In [147]:
(dfo['adj_close_price'][len(dfo)-1]-dfo['adj_close_price'][0])/dfo['adj_close_price'][0]*100

107.4283664979756

In [150]:
#test
import pandas as pd
sums=0
listna=[]
for j in range(0,11): #เเต่ละ signal 
    for i in range(0,10): #เเต่ละ data
        sums+=coll[i][0][j]
    sums=sums/10
    listna.append(sums)
tdna=listna[10]
w=listna[0:10]
dfo=pd.read_csv('TSLA_18_year_testing_signal.csv')
count1,count2,count3=0,0,0
decision_d_l=[]
il=[]
sell_idx,buy_idx=[],[]
def testing(td,w,df):
    global count1,count2,count3,sell_idx,buy_idx,sl1
    mon=100000
    amount=0
    sumall=0
    sl1=dfo['sma89_signal']
    sl2=dfo['obv_macd_signal']
    sl3=dfo['cdc_signal']
    sl4=dfo['rsi_signal']
    sl5=dfo['renko_signal']
    sl6=dfo['sar_signal']
    sl7=dfo['macd_signal']
    sl8=dfo['stoch_signal']
    sl9=dfo['obv_renko_signal']
    sl10=dfo['aroon_signal']
    
    for i in range(0,len(df)):
        decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
        if i!=0:
            if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
                    sell_idx.append(i)
        if decision_d>td:
            sumall=mon//df['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*df['adj_close_price'][i]*1.002
            count1+=1
            buy_idx.append(i)
        elif decision_d<-td:
            mon+=amount*df['adj_close_price'][i]*0.998
            amount=0
            count2+=1
            sell_idx.append(i)
        else:
            count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
    ret=mon+amount*df['adj_close_price'][len(df)-1]
    return (ret-100000)/1000
testing(tdna,w,dfo)

-3.8563480945091433

In [149]:
(dfo['adj_close_price'][len(dfo)-1]-dfo['adj_close_price'][0])/dfo['adj_close_price'][0]*100

4.161849649068813

In [151]:
listna

[0.05171901078056007,
 0.09168125927799935,
 0.0635959532800144,
 0.19770249244774873,
 0.06882487224742326,
 0.12934489843397567,
 0.07462249921901046,
 0.14327630557640067,
 0.14077705830467716,
 0.1346100853889713,
 0.1493072239607629]

In [63]:
listna

[0.03418788291218656,
 0.03149076244234548,
 0.15037019569028254,
 0.1325433769862197,
 0.14793535036282862,
 0.08853880692902165,
 0.09635943910162295,
 0.12403901528853958,
 0.12794771591086848,
 0.15992870893847594,
 0.19253667530824434]

In [ ]:
#BH 16 107.4283664979756
#back 16 110.85032925866324
#BH 4.161849649068813
#Forward 0

In [16]:
listna=[0.07209369806093725,
 0.07185038480865677,
 0.07605418723127022,
 0.046688922200022735,
 0.05800967858322942,
 0.20641780081608485,
 0.13221615163816813,
 0.17658586648082206,
 0.13257461186240724,
 0.12475782848656121,
 0.23315362570785392]

In [170]:
a=[0.07209369806093725,
 0.07185038480865677,
 0.07605418723127022,
 0.046688922200022735,
 0.05800967858322942,
 0.20641780081608485,
 0.13221615163816813,
 0.17658586648082206,
 0.13257461186240724,
 0.12475782848656121,
 0.23315362570785392]
td=a[10]
w=a[0:10]
df=splittest[0]

decision_d_l=[]
il=[]
global count1,count2,count3,sell_idx,buy_idx,sl1
mon=100000
amount=0
sumall=0
df=df.reset_index()
sl1=df['sma89_signal']
sl2=df['obv_macd_signal']
sl3=df['cdc_signal']
sl4=df['rsi_signal']
sl5=df['renko_signal']
sl6=df['sar_signal']
sl7=df['macd_signal']
sl8=df['stoch_signal']
sl9=df['obv_renko_signal']
sl10=df['aroon_signal']

for i in range(0,len(df)):
    decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
    if i!=0:
        if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                mon+=amount*df['Close'][i]*0.998
                amount=0
                sell_idx.append(i)
    if decision_d>td:
        sumall=mon//df['adj_close_price'][i]
        amount+=sumall
        mon-=sumall*df['adj_close_price'][i]*1.002
        count1+=1
        buy_idx.append(i)
    elif decision_d<-td:
        mon+=amount*df['adj_close_price'][i]*0.998
        amount=0
        count2+=1
        sell_idx.append(i)
    else:
        count3+=1
decision_d_l.append(mon+amount*df['adj_close_price'][i])
il.append(i)
ret=mon+amount*df['adj_close_price'][len(df)-1]
print (("TEST",b,ret-100000)/1000)

NameError: name 'b' is not defined

In [168]:
sl1

155    1
156    1
157    1
158   -1
159   -1
160    1
161    1
162    1
163    1
164    1
165   -1
166   -1
167   -1
168   -1
169   -1
170   -1
171   -1
172   -1
173   -1
174   -1
175   -1
176   -1
177   -1
178   -1
179   -1
180   -1
181   -1
182   -1
183   -1
184   -1
185   -1
Name: sma89_signal, dtype: int64